In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
#import csv filee interactions_train.csv
interactions = pd.read_csv('interactions_train.csv')
items = pd.read_csv('items.csv')

display(interactions.head())
display(items.head())

#limited to 100 rows
limited_items = items.head(100)

,u,i,t
0,4456,8581,1.687541e+09
1,142,1964,1.679585e+09
2,362,3705,1.706872e+09
3,1809,11317,1.673533e+09
4,4384,1323,1.681402e+09


,Title,Author,ISBN Valid,Publisher,Subjects,i
0,Classification décimale universelle : édition ...,NaN,9782871303336; 2871303339,Ed du CEFAL,Classification décimale universelle; Indexatio...,0
1,Les interactions dans l'enseignement des langu...,"Cicurel, Francine, 1947-",9782278058327; 2278058320,Didier,didactique--langue étrangère - enseignement; d...,1
2,Histoire de vie et recherche biographique : pe...,NaN,2343190194; 9782343190198,L'Harmattan,Histoires de vie en sociologie; Sciences socia...,2
3,Ce livre devrait me permettre de résoudre le c...,"Mazas, Sylvain, 1980-",9782365350020; 236535002X; 9782365350488; 2365...,Vraoum!,Moyen-Orient; Bandes dessinées autobiographiqu...,3
4,Les années glorieuses : roman /,"Lemaitre, Pierre, 1951-",9782702180815; 2702180817; 9782702183618; 2702...,Calmann-Lévy,France--1945-1975; Roman historique; Roman fra...,4


## Functions

In [3]:
def get_book_metadata(isbn):
    api_url = f"https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}"

    try:
        response = requests.get(api_url)
        response.raise_for_status()  # Exception for HTTP errors
        data = response.json()

        if 'items' in data:
            metadata = []
            for item in data['items']:
                volume_info = item['volumeInfo']
                book_data = {
                    'title': volume_info.get('title'),
                    'authors': volume_info.get('authors'),
                    'language': volume_info.get('language'),
                    'publisher': volume_info.get('publisher'),
                    'published_date': volume_info.get('publishedDate'),
                    'subjects': volume_info.get('categories')  
                }
                metadata.append(book_data)
            return metadata
        else:
            return {"error": "No books found matching that ISBN."}
    except requests.exceptions.RequestException as e:
        return {"error": f"An error occurred: {e}"}

# Example usage:
isbn = "9782702180815"
book_metadata = get_book_metadata(isbn)
book_metadata

[{'title': 'Le grand monde',
  'authors': ['Pierre Lemaitre'],
  'language': 'fr',
  'publisher': None,
  'published_date': '2022',
  'subjects': ['French literature']}]

In [4]:
#identify missing values for items
missing_values = items.isnull().sum()
missing_values

Title            0
Author        2653
ISBN Valid     723
Publisher       25
Subjects      2223
i                0
dtype: int64

In [5]:
#count the number of elements in each column
count_elements = items.count()
count_elements

Title         15291
Author        12638
ISBN Valid    14568
Publisher     15266
Subjects      13068
i             15291
dtype: int64

In [6]:
limited_items

,Title,Author,ISBN Valid,Publisher,Subjects,i
0,Classification décimale universelle : édition ...,NaN,9782871303336; 2871303339,Ed du CEFAL,Classification décimale universelle; Indexatio...,0
1,Les interactions dans l'enseignement des langu...,"Cicurel, Francine, 1947-",9782278058327; 2278058320,Didier,didactique--langue étrangère - enseignement; d...,1
2,Histoire de vie et recherche biographique : pe...,NaN,2343190194; 9782343190198,L'Harmattan,Histoires de vie en sociologie; Sciences socia...,2
3,Ce livre devrait me permettre de résoudre le c...,"Mazas, Sylvain, 1980-",9782365350020; 236535002X; 9782365350488; 2365...,Vraoum!,Moyen-Orient; Bandes dessinées autobiographiqu...,3
4,Les années glorieuses : roman /,"Lemaitre, Pierre, 1951-",9782702180815; 2702180817; 9782702183618; 2702...,Calmann-Lévy,France--1945-1975; Roman historique; Roman fra...,4
...,...,...,...,...,...,...
95,Akane banashi /,"Suenaga, Yuki 19..-....",9791032715413; 9791032715420; 9791032715437; 9...,Ki-oon,Mangas; Mangas.,95
96,Des maux à dire /,"Lema, Beatriz 1985-....",9791040802396,Sarbacane,Bandes dessinées; Mères et filles; Maladies me...,96
97,Il y a longtemps que je t'aime /,"Spénale, Marie",2203276509; 9782203276505,Casterman,Bandes dessinées,97
98,Regarde les lumières mon amour /,"Ernaux, Annie",9782370210371; 2370210370,Seuil,NaN,98


In [11]:
# Initialize a list to store metadata
limited_metadata_list = []

# Iterate over the limited_items DataFrame
for _, row in limited_items.iterrows():
    isbn = row['ISBN Valid']
    if pd.notnull(isbn):  # Ensure the ISBN is not NaN
        primary_isbn = isbn.split(';')[0]  # Use the first ISBN if multiple are listed
        metadata = get_book_metadata(primary_isbn)
        if isinstance(metadata, list):  # Ensure metadata is a list of dictionaries
            for entry in metadata:
                entry['i'] = row['i']  # Add the 'i' column value to the metadata
                entry['title'] = row['Title']  # Add the 'Title' column value to the metadata
                entry['isbn'] = row['ISBN Valid']  # Add the 'ISBN Valid' column value to the metadata
                limited_metadata_list.append(entry)

# Create a new DataFrame from the collected metadata
limited_metadata_df = pd.DataFrame(
    limited_metadata_list, 
    columns=['authors', 'publisher', 'subjects', 'language', 'published_date', 'i', 'title', 'isbn']
)

# Display the new DataFrame
limited_metadata_df.head()

,authors,publisher,subjects,language,published_date,i,title,isbn
0,[UDC Consortium (The Hague)],None,None,fr,2012,0,Classification décimale universelle : édition ...,9782871303336; 2871303339
1,[Francine Cicurel],Editions Didier,[Interaction analysis in education],fr,2011,1,Les interactions dans l'enseignement des langu...,9782278058327; 2278058320
2,"[Aneta Slowik, Hervé Breton, Gaston Pineau]",Histoire de vie et formation,[Narrative inquiry (Research method)],fr,2020,2,Histoire de vie et recherche biographique : pe...,2343190194; 9782343190198
3,[Sylvain Mazas],Vraoum !,None,fr,2012-06-07,3,Ce livre devrait me permettre de résoudre le c...,9782365350020; 236535002X; 9782365350488; 2365...
4,[Pierre Lemaitre],None,[French literature],fr,2022,4,Les années glorieuses : roman /,9782702180815; 2702180817; 9782702183618; 2702...
